In [41]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import datetime
import pandas as pd
from time import sleep
import sqlite3
import numpy as np

In [2]:
# 일자 생성 함수
def getDates():
    res = []
    start = datetime.date(2016,1,1)
    end = datetime.date(2017,12,31)
    delta = datetime.timedelta(days=1)
    
    date = start-delta
    for _ in range((end-start).days + 1):
        date = date+delta
        res.append(str(date))
    return res

In [11]:
# 프레임 접근 함수
def getFrame(date, port):
    driver.get('http://www.airportal.go.kr/life/airinfo/RbHanFrm.jsp')
    driver.find_element_by_id('current_date').clear()
    driver.find_element_by_id('current_date').send_keys(date)
    select = Select(driver.find_element_by_id('select_airport'))
    select.select_by_visible_text(port)
    driver.switch_to.frame(driver.find_element_by_name('sframe'))
    sleep(2)
    return None

In [77]:
# 데이터 수집 함수
def getData(date, port):
    column = [0,2,4,6,8,10,12,14]
    colnames = ['항공사','편명','출발지','계획','예상','도착','구분','현황']
    getFrame(date, port)
    html = driver.find_element_by_tag_name('table').get_attribute('innerHTML')
    df = pd.read_html(html)[0]
    df = df.loc[~df[0].isna(), column]
    df.columns = colnames
    df['기준년월'] = date
    df['공항명'] = port
    return np.array(df)

In [30]:
# DB 생성
conn = sqlite3.connect('Airport.db')
cur = conn.cursor()

In [35]:
# 테이블 생성
try:
    cur.execute('create table 항공정보(항공사,편명,출발지,계획,예상,도착,구분,현황,기준년월,공항명)')
except:
    pass
#cur.execute('delete from 항공정보')

In [13]:
# 드라이버 객체 생성 및 설정
options = webdriver.ChromeOptions()
# options.add_argument('headless')
# options.add_argument('disable-gpu')
driver = webdriver.Chrome('C:/Users/sangjin/chromedriver.exe', chrome_options=options)

In [14]:
# 포트 및 일자 생성
ports = ['인천','김포','청주','양양','군산','원수','김해','제주','대구','광주','여수','울산','포항','사천','무안']
dates = getDates()

In [78]:
# raw data 수집
sql = 'insert into 항공정보 values (?,?,?,?,?,?,?,?,?,?)'

res = []
for date in dates[:3]:
    for port in ports[:2]:
        cur.executemany(sql, getData(date, port))

In [82]:
# 데이터 출력
cur.execute('select * from 항공정보')
rows = cur.fetchall()
df = pd.DataFrame(rows)
colnames = ['항공사','편명','출발지','계획','예상','도착','구분','현황','기준년월','공항명']
df.columns = colnames
# writer = pd.ExcelWriter('180626_Airport.xlsx', engine='xlsxwriter')
# df.to_excel(writer,'Sheet1', index=False)
# writer.save()
# writer.close()